In [ ]:
!rm -f *.py
!wget https://raw.githubusercontent.com/artificial-intelligence-ml-cti/ml_cti/main/proiect/dale_chall.py

--2021-11-22 17:57:25--  https://raw.githubusercontent.com/artificial-intelligence-ml-cti/ml_cti/main/proiect/dale_chall.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27456 (27K) [text/plain]
Saving to: ‘dale_chall.py’

dale_chall.py       100%[===================>]  26.81K  --.-KB/s    in 0.001s  

2021-11-22 17:57:25 (24.3 MB/s) - ‘dale_chall.py’ saved [27456/27456]



In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
!rm -rf data*
!wget https://github.com/artificial-intelligence-ml-cti/ml_cti/raw/main/proiect/data.zip
!unzip data.zip

--2021-11-22 17:57:25--  https://github.com/artificial-intelligence-ml-cti/ml_cti/raw/main/proiect/data.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/artificial-intelligence-ml-cti/ml_cti/main/proiect/data.zip [following]
--2021-11-22 17:57:26--  https://raw.githubusercontent.com/artificial-intelligence-ml-cti/ml_cti/main/proiect/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 741506 (724K) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>] 724.13K  --.-KB/s    in 0.03s   

2021-11-22 17:57:26 (23.9 MB/s) - ‘data.zip’ saved [741506/741506]

Arc

In [ ]:
!echo "***\n lista fisiere :"
!ls data
!readlink -f data/

***\n lista fisiere :
test.xlsx  train.xlsx
/content/data


In [ ]:
!pip install pyphen nltk pandas sklearn openpyxl

In [ ]:
import numpy as np
import pandas as pd

from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import balanced_accuracy_score

from dale_chall import DALE_CHALL


In [ ]:
dtypes = {"sentence":"string", "token" : "string", "complexity" : "float64"}
train = pd.read_excel('/content/data/train.xlsx', dtype = dtypes, keep_default_na = False)
test = pd.read_excel('/content/data/test.xlsx', dtype = dtypes, keep_default_na = False)

print('train.data: ', train.shape)
print('test.data: ', test.shape)
print(train.columns)
print(train['corpus'].unique())
print(len(train['token'].unique()))
print(len(train))

print(train.iloc[0])

train.data:  (7662, 4)
test.data:  (1338, 3)
Index(['corpus', 'sentence', 'token', 'complex'], dtype='object')
['bible' 'biomed' 'europarl']
3487
7662
corpus                                                  bible
sentence    Behold, there came up out of the river seven c...
token                                                   river
complex                                                     0
Name: 0, dtype: object


WORD COUNTER

In [ ]:
import string

propozitii=''

for index in range(len(train)):
  propozitii = propozitii+ ' ' +train.iloc[index]['sentence']

for index in range(len(test)):
  propozitii = propozitii+ ' ' +test.iloc[index]['sentence']

punctuations = ',!?".'
nopunct=""

for char in propozitii:
  if char not in punctuations:
    nopunct = nopunct + char

propozitii = nopunct
propozitii = propozitii.lower()



def word_count(str):
  counts = dict()
  words = str.split()

  for word in words:
    if word in counts:
      counts[word] += 1
    else:
      counts[word] = 1
  return counts



lista = list(word_count(propozitii).items())
wordcount = np.array(lista)

totalcuvinte = 0

for i in range(len(wordcount)):
  totalcuvinte = totalcuvinte + wordcount[i][1].astype(np.int)

print(wordcount)
print(np.where(wordcount == 'behold'))
def numar_aparitii(word):
  if word in wordcount:
    index = np.where(wordcount == word)
    nrap = wordcount[index[0],1].astype('float64') 
    nrap = nrap.astype('float64')

    nrap[0] = round(nrap[0]*100/totalcuvinte,4)
  
 
  
    return nrap[0]
  else:
    return 0







[['behold' '118']
 ['there' '435']
 ['came' '136']
 ...
 ['rye' '1']
 ['durum' '1']
 ['c4-0292/98-98/0107(cns))ii' '1']]
(array([0]), array([0]))


In [ ]:

def is_dale_chall(word):
  return int(word in DALE_CHALL)
 
def is_title(word):
  if int(word.istitle())==0:
    return 1
  else:
    return 0


def up_letters(word):
  nr = 0
  cnt=0
  for letter in word:
    cnt +=1
    if letter.isupper():
      nr += 1
  if nr == 0:
    return 1
  else:
    return nr/10

def litere_rare(word):
  litere = 'jqzxJQZX'
  for letter in word:
    if letter in litere:
      return 1
    else:
      return 0

def corpus_feature(corpus):
  d = {"bible" : [1,0,0], "europarl" : [0,1,0], "biomed" : [0,0,1]}
  return d[corpus]

def nr_synsets(word):
  return len(wn.synsets(word))/10

def get_wordnet_features(word):
  features = []
  features.append(nr_synsets(word))
  return np.array(features)/10


def get_word_structure_features(word):
  features = []
  features.append(is_dale_chall(word))
  features.append(is_title(word))
  features.append(up_letters(word))
  features.append(litere_rare(word))
  features.append(len(word)/10)
  features.append(numar_aparitii(word))
  return np.array(features)

print(get_word_structure_features('to'))



[1.     1.     1.     0.     0.2    2.5261]


In [ ]:
def featurize(row):
  word = row['token']
  all_features=[]
  all_features.extend(get_word_structure_features(word))
  all_features.extend(get_wordnet_features(word))
  all_features.extend(corpus_feature(row['corpus']))
  return np.array(all_features)

print(featurize(train.iloc[10]))

def featurize_df(df):
  nr_of_features = len(featurize(df.iloc[0]))
  nr_of_examples = len(df)
  features = np.zeros((nr_of_examples, nr_of_features))
  for index, row in df.iterrows():
    row_ftrs = featurize(row)
    features[index, :] = row_ftrs
  return features



[0.     1.     1.     0.     0.4    0.0326 0.02   1.     0.     0.    ]


In [ ]:

x_train = featurize_df(train)
y_train = train['complex'].values

x_test = featurize_df(test)

model = KNeighborsClassifier(n_neighbors=7)
model.fit(x_train, y_train)
preds = model.predict(x_test)
print(preds)


[0 0 1 ... 0 0 0]


ACURATETE KNN

In [ ]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x_train, y_train, test_size = 0.1, random_state=42)

print(xtrain.shape)
print(xtest.shape)
print(np.bincount(ytrain))
print(np.bincount(ytest))
for index in range (1,21):
  model = KNeighborsClassifier(n_neighbors=index)
  model.fit(xtrain, ytrain)
  predz = model.predict(xtest)

  print(balanced_accuracy_score(ytest,predz))

model = KNeighborsClassifier(n_neighbors=1)
model.fit(x_train, y_train)
preds = model.predict(x_test)

(6895, 10)
(767, 10)
[6219  676]
[693  74]
0.7624020124020123
0.6834171834171834
0.7184489684489684
0.6723314223314223
0.6728579228579228
0.6535041535041535
0.6687239187239187
0.6453043953043953
0.6438613938613938
0.6407121407121407
0.6371046371046372
0.6296263796263797
0.6371046371046372
0.6204418704418704
0.6325123825123825
0.6151281151281152
0.6204418704418704
0.6158496158496158
0.6346768846768847
0.6226063726063726


In [ ]:
df = pd.DataFrame()
df['id'] = test.index + len(train) + 1
df['complex'] = preds
df.to_csv('submission.csv', index=False)